<a href="https://colab.research.google.com/github/guscldns/TestProject/blob/main/0713/13_1_paper_implementation(GoogLeNet).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GoogleNet 구조

In [8]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

import matplotlib.pyplot as plt

In [71]:
def create_googlenet():
    # creates GoogLeNet a.k.a. Inception v1 (Szegedy, 2015)
    input = tf.keras.Input(shape=(224, 224, 3))
    conv1_7x7_s2 = tf.keras.layers.Conv2D(64,(7,7), padding = 'same', strides = 2, activation = 'relu', name = 'conv1_7x7_s2')(input)
    pool1_3x3_s2 = tf.keras.layers.MaxPool2D((3,3), strides = 2, padding = 'same', name ='pool1/3x3_s2')(conv1_7x7_s2)
    pool1_norm1  = tf.nn.local_response_normalization(pool1_3x3_s2)

    conv2_3x3_reduce = tf.keras.layers.Conv2D(64,(1,1), padding = 'same', strides = 1, activation = 'relu', name='conv2/3x3_reduce')(pool1_norm1)
    conv2_3x3 = tf.keras.layers.Conv2D(192,(3,3), padding = 'same', strides = 1, activation = 'relu', name = 'conv2_3x3')(conv2_3x3_reduce)
    conv2_norm2 = tf.nn.local_response_normalization(conv2_3x3)


    # Inception 3a ############################################################
    pool2_3x3_s2 = tf.keras.layers.MaxPool2D((3,3), strides = 2, padding = 'same', name ='pool2_3x3_s2')(conv2_norm2)

    inception_3a_3x3_reduce = tf.keras.layers.Conv2D(96,(1,1), padding = 'same', strides = 1, activation = 'relu', name = 'inception_3a_3x3_reduce')(pool2_3x3_s2)
    inception_3a_5x5_reduce = tf.keras.layers.Conv2D(16,(1,1), padding = 'same', strides = 1, activation = 'relu', name = 'inception_3a_5x5_reduce')(pool2_3x3_s2)
    inception_3a_pool = tf.keras.layers.MaxPool2D((3,3), strides = 1, padding = 'same', name = 'inception_3a_pool')(pool2_3x3_s2)
    inception_3a_1x1 = tf.keras.layers.Conv2D(64,(1,1), padding = 'same', strides = 1, activation = 'relu', name='inception_3a_1x1')(pool2_3x3_s2)

    inception_3a_3x3 = tf.keras.layers.Conv2D(128,(3,3), padding = 'same', strides = 1, activation = 'relu', name = 'inception_3a_3x3')(inception_3a_3x3_reduce)
    inception_3a_5x5 = tf.keras.layers.Conv2D(32,(5,5), padding = 'same', strides = 1, activation = 'relu', name = 'inception_3a_5x5')(inception_3a_5x5_reduce)
    inception_3a_pool_proj = tf.keras.layers.Conv2D(32,(1,1), padding = 'same', strides = 1, activation = 'relu', name = 'inception_3a_pool_proj')(inception_3a_pool)

    # output : concat
    inception_3a_output = layers.Concatenate(axis=-1,
                                      name='inception_3a/output')([inception_3a_1x1,inception_3a_3x3,
                                                                   inception_3a_5x5,inception_3a_pool_proj])

    # Inception 3b ############################################################
    inception_3b_3x3_reduce = tf.keras.layers.Conv2D(128,(1,1), padding = 'same', strides = 1, activation = 'relu', name = 'inception_3b_3x3_reduce')(inception_3a_output)
    inception_3b_5x5_reduce = tf.keras.layers.Conv2D(32,(1,1), padding = 'same', strides = 1, activation = 'relu', name = 'inception_3b_5x5_reduce')(inception_3a_output)
    inception_3b_pool = tf.keras.layers.MaxPool2D((3,3), strides = 1, padding = 'same', name = 'inception_3b_pool')(inception_3a_output)
    inception_3b_1x1 = tf.keras.layers.Conv2D(128,(1,1), padding = 'same', strides = 1, activation = 'relu', name='inception_3b_1x1')(inception_3a_output)

    inception_3b_3x3 = tf.keras.layers.Conv2D(192,(3,3), padding = 'same', strides = 1, activation = 'relu', name = 'inception_3b_3x3')(inception_3b_3x3_reduce)
    inception_3b_5x5 = tf.keras.layers.Conv2D(96,(5,5), padding = 'same', strides = 1, activation = 'relu', name = 'inception_3b_5x5')(inception_3b_5x5_reduce)
    inception_3b_pool_proj = tf.keras.layers.Conv2D(64,(1,1), padding = 'same', strides = 1, activation = 'relu', name = 'inception_3b_pool_proj')(inception_3b_pool)

    # output : concat
    inception_3b_output = layers.Concatenate(axis=-1,
                                      name='inception_3b/output')([inception_3b_1x1,inception_3b_3x3,
                                                                   inception_3b_5x5,inception_3b_pool_proj])

    ###########################################################################
    # Inception 4a ############################################################
    pool3_3x3_s2 = tf.keras.layers.MaxPool2D((3,3), strides =2, padding = 'same', name ='pool3_3x3_s2')(inception_3b_output)

    inception_4a_3x3_reduce = tf.keras.layers.Conv2D(96,(1,1), padding = 'same', strides = 1, activation = 'relu')(pool3_3x3_s2)
    inception_4a_5x5_reduce = tf.keras.layers.Conv2D(16,(1,1), padding = 'same', strides = 1, activation = 'relu')(pool3_3x3_s2)
    inception_4a_pool = tf.keras.layers.MaxPool2D((3,3), strides = 1, padding = 'same')(pool3_3x3_s2)
    inception_4a_1x1 = tf.keras.layers.Conv2D(192,(1,1), padding = 'same', strides = 1, activation = 'relu')(pool3_3x3_s2)

    inception_4a_3x3 = tf.keras.layers.Conv2D(208,(3,3), padding = 'same', strides = 1, activation = 'relu')(inception_4a_3x3_reduce)
    inception_4a_5x5 = tf.keras.layers.Conv2D(48,(5,5), padding = 'same', strides = 1, activation = 'relu')(inception_4a_5x5_reduce)
    inception_4a_pool_proj = tf.keras.layers.Conv2D(64,(1,1), padding = 'same', strides = 1, activation = 'relu')(inception_4a_pool)

    # output : concat
    inception_4a_output = layers.Concatenate(axis=-1,
                                      name='inception_4a/output')([inception_4a_1x1,inception_4a_3x3,
                                                                   inception_4a_5x5,inception_4a_pool_proj])

    # auxiliary classifier **********************
    loss1_avg_pool = tf.keras.layers.AveragePooling2D((3,3), strides = 3, padding = 'valid')(inception_4a_output)
    loss1_conv = tf.keras.layers.Conv2D(128,(1,1), padding = 'same', strides = 1, activation = 'relu')(loss1_avg_pool)
    flatten_3 = tf.keras.layers.Flatten()(loss1_conv)
    loss1_fc = tf.keras.layers.Dense(1024, activation = 'relu')(flatten_3)
    dropout_3 = tf.keras.layers.Dropout(0.7)(loss1_fc)
    loss1_classifier = tf.keras.layers.Dense(1000, activation = 'softmax')(dropout_3)

    # Inception 4b ############################################################
    inception_4b_3x3_reduce = tf.keras.layers.Conv2D(112,(1,1), padding = 'same', strides = 1, activation = 'relu')(inception_4a_output)
    inception_4b_5x5_reduce = tf.keras.layers.Conv2D(24,(1,1), padding = 'same', strides = 1, activation = 'relu')(inception_4a_output)
    inception_4b_pool = tf.keras.layers.MaxPool2D((3,3), strides = 1, padding = 'same')(inception_4a_output)
    inception_4b_1x1 = tf.keras.layers.Conv2D(160,(1,1), padding = 'same', strides = 1, activation = 'relu')(inception_4a_output)

    inception_4b_3x3 = tf.keras.layers.Conv2D(224,(3,3), padding = 'same', strides = 1, activation = 'relu')(inception_4b_3x3_reduce)
    inception_4b_5x5 = tf.keras.layers.Conv2D(64,(5,5), padding = 'same', strides = 1, activation = 'relu')(inception_4b_5x5_reduce)
    inception_4b_pool_proj = tf.keras.layers.Conv2D(64,(1,1), padding = 'same', strides = 1, activation = 'relu')(inception_4b_pool)

    # output : concat
    inception_4b_output = layers.Concatenate(axis=-1,
                                      name='inception_4b/output')([inception_4b_1x1,inception_4b_3x3,
                                                                   inception_4b_5x5,inception_4b_pool_proj])


    # Inception 4c ############################################################
    inception_4c_3x3_reduce = tf.keras.layers.Conv2D(128,(1,1), padding = 'same', strides = 1, activation = 'relu')(inception_4b_output)
    inception_4c_5x5_reduce = tf.keras.layers.Conv2D(24,(1,1), padding = 'same', strides = 1, activation = 'relu')(inception_4b_output)
    inception_4c_pool = tf.keras.layers.MaxPool2D((3,3), strides = 1, padding = 'same')(inception_4b_output)
    inception_4c_1x1 = tf.keras.layers.Conv2D(128,(1,1), padding = 'same', strides = 1, activation = 'relu')(inception_4b_output)

    inception_4c_3x3 = tf.keras.layers.Conv2D(256,(3,3), padding = 'same', strides = 1, activation = 'relu')(inception_4c_3x3_reduce)
    inception_4c_5x5 = tf.keras.layers.Conv2D(64,(5,5), padding = 'same', strides = 1, activation = 'relu')(inception_4c_5x5_reduce)
    inception_4c_pool_proj = tf.keras.layers.Conv2D(64,(1,1), padding = 'same', strides = 1, activation = 'relu')(inception_4c_pool)

    # output : concat
    inception_4c_output = layers.Concatenate(axis=-1,
                                      name='inception_4c/output')([inception_4c_1x1,inception_4c_3x3,
                                                                   inception_4c_5x5,inception_4c_pool_proj])

    # Inception 4d ############################################################
    inception_4d_3x3_reduce = tf.keras.layers.Conv2D(144,(1,1), padding = 'same', strides = 1, activation = 'relu')(inception_4c_output)
    inception_4d_5x5_reduce = tf.keras.layers.Conv2D(32,(1,1), padding = 'same', strides = 1, activation = 'relu')(inception_4c_output)
    inception_4d_pool = tf.keras.layers.MaxPool2D((3,3), strides = 1, padding = 'same')(inception_4c_output)
    inception_4d_1x1 = tf.keras.layers.Conv2D(112,(1,1), padding = 'same', strides = 1, activation = 'relu')(inception_4c_output)

    inception_4d_3x3 = tf.keras.layers.Conv2D(288,(3,3), padding = 'same', strides = 1, activation = 'relu')(inception_4d_3x3_reduce)
    inception_4d_5x5 = tf.keras.layers.Conv2D(64,(5,5), padding = 'same', strides = 1, activation = 'relu')(inception_4d_5x5_reduce)
    inception_4d_pool_proj = tf.keras.layers.Conv2D(64,(1,1), padding = 'same', strides = 1, activation = 'relu')(inception_4d_pool)

    # output : concat
    inception_4d_output = layers.Concatenate(axis=-1,
                                      name='inception_4d/output')([inception_4d_1x1,inception_4d_3x3,
                                                                   inception_4d_5x5,inception_4d_pool_proj])

    # auxiliary classifier **********************
    loss2_avg_pool = tf.keras.layers.AveragePooling2D((3,3), strides = 3, padding = 'valid')(inception_4d_output)
    loss2_conv = tf.keras.layers.Conv2D(128,(1,1), padding = 'same', strides = 1, activation = 'relu')(loss2_avg_pool)
    flatten_4 = tf.keras.layers.Flatten()(loss2_conv)
    loss2_fc = tf.keras.layers.Dense(1024, activation = 'relu')(flatten_4)
    dropout_4 = tf.keras.layers.Dropout(0.7)(loss2_fc)
    loss2_classifier = tf.keras.layers.Dense(1000, activation = 'softmax')(dropout_4)

    # Inception 4e ############################################################
    inception_4e_3x3_reduce = tf.keras.layers.Conv2D(160,(1,1), padding = 'same', strides = 1, activation = 'relu')(inception_4d_output)
    inception_4e_5x5_reduce = tf.keras.layers.Conv2D(32,(1,1), padding = 'same', strides = 1, activation = 'relu')(inception_4d_output)
    inception_4e_pool = tf.keras.layers.MaxPool2D((3,3), strides = 1, padding = 'same')(inception_4d_output)
    inception_4e_1x1 = tf.keras.layers.Conv2D(256,(1,1), padding = 'same', strides = 1, activation = 'relu')(inception_4d_output)

    inception_4e_3x3 = tf.keras.layers.Conv2D(320,(3,3), padding = 'same', strides = 1, activation = 'relu')(inception_4e_3x3_reduce)
    inception_4e_5x5 = tf.keras.layers.Conv2D(128,(5,5), padding = 'same', strides = 1, activation = 'relu')(inception_4e_5x5_reduce)
    inception_4e_pool_proj = tf.keras.layers.Conv2D(128,(1,1), padding = 'same', strides = 1, activation = 'relu')(inception_4e_pool)

    # output : concat
    inception_4e_output = layers.Concatenate(axis=-1,
                                      name='inception_4e/output')([inception_4e_1x1,inception_4e_3x3,
                                                                   inception_4e_5x5,inception_4e_pool_proj])

    ###########################################################################
    # Inception 5a ############################################################
    pool4_3x3_s2 = tf.keras.layers.MaxPool2D((3,3), strides = 2, padding = 'same')(inception_4e_output)

    inception_5a_3x3_reduce = tf.keras.layers.Conv2D(160,(1,1), padding = 'same', strides = 1, activation = 'relu')(pool4_3x3_s2)
    inception_5a_5x5_reduce = tf.keras.layers.Conv2D(32,(1,1), padding = 'same', strides = 1, activation = 'relu')(pool4_3x3_s2)
    inception_5a_pool = tf.keras.layers.MaxPool2D((3,3), strides = 1, padding = 'same')(pool4_3x3_s2)
    inception_5a_1x1 = tf.keras.layers.Conv2D(256,(1,1), padding = 'same', strides = 1, activation = 'relu')(pool4_3x3_s2)

    inception_5a_3x3 = tf.keras.layers.Conv2D(320,(3,3), padding = 'same', strides = 1, activation = 'relu')(inception_5a_3x3_reduce)
    inception_5a_5x5 = tf.keras.layers.Conv2D(128,(5,5), padding = 'same', strides = 1, activation = 'relu')(inception_5a_5x5_reduce)
    inception_5a_pool_proj = tf.keras.layers.Conv2D(128,(1,1), padding = 'same', strides = 1, activation = 'relu')(inception_5a_pool)

    # output : concat
    inception_5a_output = layers.Concatenate(axis=-1,
                                      name='inception_5a/output')([inception_5a_1x1,inception_5a_3x3,
                                                                   inception_5a_5x5,inception_5a_pool_proj])

    # Inception 5b ############################################################
    inception_5b_3x3_reduce = tf.keras.layers.Conv2D(192,(1,1), padding = 'same', strides = 1, activation = 'relu')(inception_5a_output)
    inception_5b_5x5_reduce = tf.keras.layers.Conv2D(48,(1,1), padding = 'same', strides = 1, activation = 'relu')(inception_5a_output)
    inception_5b_pool = tf.keras.layers.MaxPool2D((3,3), strides = 1, padding = 'same')(inception_5a_output)
    inception_5b_1x1 = tf.keras.layers.Conv2D(384,(1,1), padding = 'same', strides = 1, activation = 'relu')(inception_5a_output)

    inception_5b_3x3 = tf.keras.layers.Conv2D(384,(3,3), padding = 'same', strides = 1, activation = 'relu')(inception_5b_3x3_reduce)
    inception_5b_5x5 = tf.keras.layers.Conv2D(128,(5,5), padding = 'same', strides = 1, activation = 'relu')(inception_5b_5x5_reduce)
    inception_5b_pool_proj = tf.keras.layers.Conv2D(128,(1,1), padding = 'same', strides = 1, activation = 'relu')(inception_5b_pool)

    # output : concat
    inception_5b_output = layers.Concatenate(axis=-1,
                                      name='inception_5b/output')([inception_5b_1x1,inception_5b_3x3,
                                                                   inception_5b_5x5,inception_5b_pool_proj])

    pool5_7x7_s1 = layers.AveragePooling2D(pool_size=(7,7), strides=(1,1), name='pool5/7x7_s2')(inception_5b_output)
    flatten_5 = tf.keras.layers.Flatten()(pool5_7x7_s1)
    dropout_5 = tf.keras.layers.Dropout(0.4)(flatten_5)
    loss3_classifier = tf.keras.layers.Dense(1000, activation = 'softmax')(dropout_5)

   # Final Model : input + output[aux.cls1, aux.cls2, main.cls]
    googlenet = tf.keras.Model(inputs=input, outputs=[loss1_classifier,loss2_classifier,loss3_classifier])

    return googlenet

In [72]:
model = create_googlenet()

In [73]:
model.summary()

Model: "model_11"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_25 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_7x7_s2 (Conv2D)          (None, 112, 112, 64  9472        ['input_25[0][0]']               
                                )                                                                 
                                                                                                  
 pool1/3x3_s2 (MaxPooling2D)    (None, 56, 56, 64)   0           ['conv1_7x7_s2[0][0]']           
                                                                                           